# Baseline Model

## Import Packages

In [1]:
import os
import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm
from sklearn.model_selection import train_test_split

import segmentation_models_pytorch as smp
from segmentation_models_pytorch.losses import DiceLoss

import albumentations as A
import cv2
from datetime import datetime, timezone, timedelta

## CONFIG

### Data Directory

In [2]:
# Project Directory
PROJECT_DIR = '/workspace/Competition/map_segmentation'
os.chdir(PROJECT_DIR)

# Data Directory
DATA_DIR = os.path.join(PROJECT_DIR, 'data','final') # Folder containing train and test data
TRAIN_DIR = os.path.join(DATA_DIR, 'train') # Folder containing train data
TRAIN_IMG_DIR = os.path.join(TRAIN_DIR, 'images') # Folder containing train images
TRAIN_MASK_DIR = os.path.join(TRAIN_DIR, 'masks') # Folder containing train masks
TRAIN_CSV_FILE = os.path.join(TRAIN_DIR, 'traindf.csv') # Folder containing train key

### Result Directory

In [3]:
# Make unique serial number from current time
kst = timezone(timedelta(hours=9))        
train_serial = datetime.now(tz=kst).strftime("%Y%m%d_%H%M%S")

# Directory to save results
RECORDER_DIR = os.path.join(PROJECT_DIR, 'results', 'train', train_serial)
os.makedirs(RECORDER_DIR, exist_ok=True)    

### Set Seed

In [4]:
RANDOM_SEED = 2022

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

### Set device

In [5]:
os.environ['CUDA_VISIBLE_DEVICES']="0"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Set hyperparameters

In [6]:
EPOCHS = 20
BATCH_SIZE = 8
LEARNING_RATE = 0.003
EARLY_STOPPING_PATIENCE = 10
IMG_SIZE = 512

ENCODER = 'timm-efficientnet-b0' # Encoder model to be used
WEIGHTS = 'imagenet' # Dataset used for pretraining

## Define Dataset

In [7]:
class SegDataset(Dataset):
    def __init__(self, df, augmentations, img_dir, mask_dir):
        self.df = df
        self.augmentations = augmentations
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = os.path.join(self.img_dir,row['img'])
        mask_path = os.path.join(self.mask_dir, row['mask'])
        
        # Load image and mask
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = np.expand_dims(mask, axis=-1)
        
        # Apply augmentations
        if self.augmentations:
            data = self.augmentations(image=image, mask=mask)
            image = data['image']
            mask = data['mask']
        
        # Reshape data
        image = np.transpose(image, (2,0,1)).astype(np.float32)
        mask = np.transpose(mask, (2,0,1)).astype(np.float32)
        
        # Image normalization
        image = torch.Tensor(image) / 255.0
        mask = torch.round(torch.Tensor(mask)/255.0)
        
        return image, mask

## Define Model

In [8]:
class SegModel(nn.Module):
    def __init__(self):
        super(SegModel, self).__init__()
        
        self.backbone = smp.Unet(
            encoder_name = ENCODER,
            encoder_weights = WEIGHTS,
            in_channels = 3, 
            classes = 1,
            activation = None
        )
        
    def forward(self, images):
        logits = self.backbone(images)
        
        return logits

## Utils

### Functions for data augmentation

In [9]:
def get_train_augs():
    return A.Compose([
        A.Resize(IMG_SIZE, IMG_SIZE),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5)
    ])

def get_valid_augs():
    return A.Compose([
        A.Resize(IMG_SIZE, IMG_SIZE)
    ])

### Train Function

In [10]:
def train_fn(dataloader, model, optimizer, loss_fn):
    model.train()
    
    total_loss = 0.0
    
    for images,masks in tqdm(dataloader):
        images = images.to(DEVICE)
        masks = masks.to(DEVICE)
        
        optimizer.zero_grad()
        logits = model(images)
        loss = loss_fn(logits, masks)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    return total_loss/len(dataloader)

### Validation function

In [11]:
def valid_fn(dataloader, model, loss_fn):
    model.eval()
    
    total_loss = 0.0
    
    with torch.no_grad():
        for images,masks in tqdm(dataloader):
            images = images.to(DEVICE)
            masks = masks.to(DEVICE)
            logits = model(images)
            loss = loss_fn(logits, masks)
            total_loss += loss.item()
    return total_loss/len(dataloader)

## Train the Model

### Instantiate Dataset & Dataloader

In [12]:
# Train data key
entiredf = pd.read_csv(TRAIN_CSV_FILE)

# Split data into train and validation datasets
traindf, validdf = train_test_split(entiredf, test_size=0.2)
traindf = traindf.reset_index(drop=True)
validdf = validdf.reset_index(drop=True)

# Make datasets and dataloaders
train_dataset = SegDataset(traindf, get_train_augs(), TRAIN_IMG_DIR, TRAIN_MASK_DIR)
valid_dataset = SegDataset(validdf, get_valid_augs(), TRAIN_IMG_DIR, TRAIN_MASK_DIR)
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size = BATCH_SIZE)

### Set model, loss function, optimizer

In [13]:
model = SegModel().to(DEVICE) 
loss_fn = DiceLoss(mode = 'binary') 
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE) 

### Train epochs

In [14]:
best_loss = np.Inf

for i in range(EPOCHS):
    train_loss = train_fn(train_loader, model, optimizer, loss_fn)
    valid_loss = valid_fn(valid_loader, model, loss_fn)
    
    # Save model if loss decreases
    if valid_loss < best_loss:
        torch.save(model.state_dict(), os.path.join(RECORDER_DIR, "best-model.pt"))
        print('saved model')
        best_loss = valid_loss
        print(f"Epoch: {i+1}, Train Loss: {train_loss} Valid Loss: {valid_loss}")

100%|██████████| 211/211 [00:39<00:00,  5.30it/s]

saved model
Epoch: 1, Train Loss: 0.24479317643863288 Valid Loss: 0.1882980796398145


## Inference

### Function to convert Mask to RLE

In [15]:
def mask_to_rle(mask):
    flatten_mask = mask.flatten()
    if flatten_mask.max() == 0:
        return f'0 {len(flatten_mask)}'
    idx = np.where(flatten_mask!=0)[0]
    steps = idx[1:]-idx[:-1]
    new_coord = []
    step_idx = np.where(np.array(steps)!=1)[0]
    start = np.append(idx[0], idx[step_idx+1])
    end = np.append(idx[step_idx], idx[-1])
    length = end - start + 1
    for i in range(len(start)):
        new_coord.append(start[i])
        new_coord.append(length[i])
    new_coord_str = ' '.join(map(str, new_coord))
    return new_coord_str

### Define Test Dataset

In [16]:
class TestDataset(Dataset):
    def __init__(self, df, img_dir):
        self.df = df
        self.img_dir = img_dir
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        imname = row['img']
        image_path = os.path.join(self.img_dir,imname)
        
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = np.transpose(image, (2,0,1)).astype(np.float32)
        image = torch.Tensor(image) / 255.0
        
        return image,imname

### Set test dataset

In [17]:
TEST_DIR = os.path.join(DATA_DIR, 'test') 
TEST_IMG_DIR = os.path.join(TEST_DIR, 'images') 
TEST_CSV_FILE = os.path.join(TEST_DIR, 'testdf.csv')

testdf = pd.read_csv(TEST_CSV_FILE)
test_dataset = TestDataset(testdf, TEST_IMG_DIR)
test_loader = DataLoader(dataset=test_dataset, batch_size=1,shuffle=False)

### Load trained model parameters

In [18]:
model.load_state_dict(torch.load(os.path.join(RECORDER_DIR, 'best-model.pt')))

<All keys matched successfully>

### Make predictions

In [19]:
file_list = [] 
pred_list = [] 
class_list = [] 

model.eval()
with torch.no_grad():
    for batch_index, (image,imname) in tqdm(enumerate(test_loader)):
        image = image.to(DEVICE)
        logit_mask = model(image)
        pred_mask = torch.sigmoid(logit_mask) 
        pred_mask = (pred_mask > 0.5) * 1.0 
        pred_rle = mask_to_rle(pred_mask.detach().cpu().squeeze(0))
        pred_list.append(pred_rle)
        file_list.append(imname[0])
        class_list.append("building")

2120it [01:25, 24.84it/s]


### Save predictions in format of sample submission file

In [20]:
# Make dataframe of predictions
results = pd.DataFrame({'img_id':file_list,'class':class_list,'prediction':pred_list})

# Change order of predictions to match that of sample_submission
sampledf = pd.read_csv(os.path.join(TEST_DIR, 'sample_submission.csv'))
sorter = list(sampledf['img_id'])
results = results.set_index('img_id')
results = results.loc[sorter].reset_index()
                       
# Save predictions
results.to_csv(os.path.join(RECORDER_DIR, 'prediction.csv'), index=False)